# Module 3.7 — Survival Curves: Who Makes It Through Variance
### Intent:
Reframe success under uncertainty as survival through variance, not relative performance or profitability.

### Core question:
> If everyone has the same expectancy, why do so few remain active long enough to realize it?

### Principle:
Survival precedes optimization. A system you abandon has no expectancy.

In [199]:
import numpy as np
import matplotlib.pyplot as plt

#np.random.seed(42)

In [200]:
n_agents = 1000
n_trades = 500

wins = 275
losses = n_trades - wins

outcomes = np.array([1]*wins + [-1]*losses)
paths = np.array(
    [np.random.permutation(outcomes) for _ in range(n_agents)]
)
equity = np.cumsum(paths, axis = 1)

# We define multiple failure criteria independently.

## (A) Capital Failure

In [201]:
capital_threshold = -20
capital_failure = np.any(equity <= capital_threshold, axis = 1)

## (B) Drawdown Duration Failure(Patience)

In [202]:
running_peak = np.maximum.accumulate(equity, axis = 1)
underwater = (equity < running_peak).astype(int)

max_underwater = []

for row in underwater:
    row_max = 0
    count = 0
   
    for i in row:
        if i == 1:
            count += 1
            row_max = max(row_max, count)
        elif count > 0:
            count = 0

    max_underwater.append(row_max)

max_underwater = np.asarray(max_underwater)
patience_threshold = n_trades * 0.25  # ~ quarter of the total trades
patience_failure = max_underwater >= patience_threshold

## (C) Confidence Failure (Psychological)
deep loss into the very 1st quarter

In [203]:
early_window = int(n_trades * 0.25)
confidence_threshold = -15

confidence_failure  = np.min(equity[:, :early_window] , axis = 1) <= confidence_threshold

In [204]:
print(f"capital_failure = {capital_failure.sum()}")
print(f"patience_failure = {patience_failure.sum()}")
print(f"confidence_failure = {confidence_failure.sum()}")

capital_failure = 3
patience_failure = 365
confidence_failure = 11


In [205]:
survived = ~(
    capital_failure |
    patience_failure |
    confidence_failure
)

print(f"out of 1000 traders only {survived.sum()} traders survived even with the winning system")

out of 1000 traders only 632 traders survived even with the winning system


# Observations
- Capital-based failure is rare in a system with positive expectancy.
- Early sharp losses eliminate some agents, but they are not the dominant exit mechanism.
- The most common cause of failure is extended time spent below prior equity highs.
- Long, shallow drawdowns lead to abandonment more often than fast, deep losses.
- Survival depends more on tolerance for stagnation than tolerance for pain.

## Interpretation:
In uncertain systems, participants do not fail because losses are unbearable, but because progress is absent for too long.